<a href="https://colab.research.google.com/github/cripedel/DeepLearning_projects/blob/DeepLearning_projects/Pokedex_Cristobal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

POKEMON-Proyecto Pokedex

Información y consejos:
Desarrollo del algoritmo empleando Keras.
• Subir datos a Google Drive y montar dicha unidad en Colab para acceder a los mismos.
•Crear vector de etiquetas (y_label) a partir del nombre de los directorios en los que
están contenidos los datos.
• Re-escalar imágenes y normalizarlas (valores de pixels entre 0 y 1). División del
conjunto de datos en training (80%) y validación (20%).
• Debido al escaso número de muestras de cada clase será necesario el uso de la técnica
de generación sintética de datos: Data augmentation.
• Como primera aproximación se puede entrenar la arquitectura de red convolucional
usada en la práctica 4 para clasificar el dataset CIFAR10.
• A partir de los resultados obtenidos (benchmark) ir aumentando la profundidad de la
red y quedarse con el modelo que presente los mejores resultados.
• Re-entrenar arquitecturas de red ya existentes en la literatura. Comparar los
resultados obtenidos con los registrados en el punto anterior.
• Evaluar el modelo con nuevas imágenes que no se encuentren en el conjuntos de
datos de entrenamiento

In [ ]:
#Importamos librerias
import numpy as np
from keras import backend as K
from keras.layers.convolutional import Conv2D
from keras.layers import Input
from keras.models import Model
from keras.layers.core import Activation, Flatten, Dense, Dropout
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import MaxPooling2D
from keras.models import Sequential
from keras.optimizers import SGD
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from google.colab import drive
from keras.preprocessing.image import ImageDataGenerator
import cv2

#Importamos datos
from google.colab import drive 
drive.mount('/content/drive')
path_pokemon = '/content/drive/My Drive/Curso_DL_CFP/Proyecto_Pokedex/DatosPokedex/dataset'
modeloentrenado = '/content/drive/My Drive/Curso_DL_CFP/Proyecto_Pokedex'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls '/content/gdrive/My Drive/Curso_DL_CFP/Proyecto_Pokedex/DatosPokedex/dataset'

bulbasaur  charmander  mewtwo  pikachu	squirtle


In [ ]:
#Creacion de las etiquetas ,buscar info  ==> image data generator ,flow from directory

datagen_train = ImageDataGenerator(rotation_range=int(180*0.1), width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True, validation_split=0.2)
val_datagen = datagen_train.flow_from_directory(path_pokemon, batch_size=32,  target_size=(32,32), class_mode='categorical', subset='validation')


train = train_datagen.flow_from_directory(
        path_pokemon,
        target_size=(32, 32),
        batch_size=32,
        class_mode='categorical')

labels = list(train.class_indices.keys())


Found 230 images belonging to 5 classes.
Found 1167 images belonging to 5 classes.


In [ ]:
#Creamos y ejecutamos la red
def deep_CNN(width, height, depth, classes,batchNorm):
  # Definimos el modo API Sequential y las dimensiones de la entrada (suponemos TF->"channels last")
  
  inputs = Input(shape=(height, width, depth)) #(X)
  
  # Definimos la arquitectura
  # Primer set de capas CONV => RELU => CONV => RELU => POOL
  x1 = Conv2D(256, (3, 3), padding="same", activation="relu")(inputs) #(X)
  if batchNorm: 
    x1 = BatchNormalization()(x1) #(X)
  x1 = Conv2D(256, (3, 3), padding="same", activation="relu")(x1) #(X)
  if batchNorm:
    x1 = BatchNormalization()(x1) #(X)
  x1 = MaxPooling2D(pool_size=(2, 2))(x1) #(X)
  x1 = Dropout(0.25)(x1) #(X)
  
  # Segundo set de capas CONV => RELU => CONV => RELU => POOL
  x2 = Conv2D(64, (3, 3), padding="same", activation="relu")(x1) #(X)
  if batchNorm: #(X)
    x2 = BatchNormalization()(x2) #(X)
  x2 = Conv2D(64, (3, 3), padding="same", activation="relu")(x2) #(X)
  if batchNorm:
    x2 = BatchNormalization()(x2) #(X)
  x2 = MaxPooling2D(pool_size=(2, 2))(x2) #(X)
  x2 = Dropout(0.25)(x2) #(X)
  
  # Primer (y único) set de capas FC => RELU
  xfc = Flatten()(x2) #(X)
  xfc = Dense(512, activation="relu")(xfc) #(X)
  if batchNorm:
    xfc = BatchNormalization()(xfc) #(X)
  xfc = Dropout(0.5)(xfc) #(X)
  # Clasificador softmax
  predictions = Dense(classes, activation="softmax")(xfc) #(X)
  
  # Unimos las entradas y el modelo mediante la función Model con parámetros inputs y ouputs (Consultar la documentación)
  model = Model(inputs=inputs, outputs=predictions) #(X)
  
  # La función debe devolver el modelo como salida           
  return model


model = deep_CNN(width=256, height=256, depth=3, classes=5, batchNorm=True) #(X)
# Compilamos el modelo
model.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.01, decay=0.01/50, momentum=0.9, nesterov=True), metrics=["accuracy"]) #(X)

# Entrenamiento de la red
print("[INFO]: Entrenando la red...")
H = model.fit_generator(train, validation_data = val_datagen, epochs=50, verbose=1) #(X)

# Evaluación del modelo
print("[INFO]: Evaluando el modelo...")
# Efectuamos la predicción (empleamos el mismo valor de batch_size que en training)
predictions = model.predict(val_datagen) #(X)
# Sacamos el report para test
print(classification_report(val_datagen.classes, predictions.argmax(axis=1), target_names=labels)) #(X)

#guardamos el modelo en drive para usarlo despues
#model.save(myModelPath+'model.pokedex.h5')

# Gráficas
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 50), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 50), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 50), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 50), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()


[INFO]: Entrenando la red...
Epoch 1/50


/usr/local/lib/python3.6/dist-packages/PIL/Image.py:989: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


ResourceExhaustedError: ignored

In [ ]:
!ls '/content/drive/My Drive/Curso_DL_CFP/Proyecto_Pokedex/'

charmander.jpg	mewtwo.jpg			     Pokedex_Cristobal.ipynb
DatosPokedex	Nivel_Basico_Desarrollo_Pokedex.zip


In [ ]:
modeloentrenado = '/content/drive/My Drive/Curso_DL_CFP/Proyecto_Pokedex/'
model.save(modeloentrenado+'modelpokedex.h5')

In [ ]:

drive.mount('/content/gdrive')
image ="/content/gdrive/My Drive/Curso_DL_CFP/Proyecto_Pokedex/charmander.jpg"

def predict_image(image, model, gt_str):
  # Creamos una copia sobre la que mostraremos el resultado (comando image.copy())
  output = image.copy() #(X)
  # Expandimos las dimensiones (32, 32, 3) a (1, 32, 32, 3)
  image = np.expand_dims(image, axis=0) #(X)

  # Clasificación de la imagen empleando el modelo
  print("[INFO]: Clasificando imagen...")
  # Realizamos la predicción
  proba = model.predict(image) #(X)
  print(proba)
  # Nos quedamos con la clase que presente una probabilidad mayor y buscamos la etiqueta en el vector labelNames
  idx = np.argmax(proba) #(X)
  label = labels[idx] #(X)
  #gt = labelNames[testY[num_img][0]] #(X)
  # En caso que en la variable gt_str no me pasen el string "CIFAR10" es que me estan pasando el string con la etiqueta
  # Si ese es el caso almaceno el gt de ese String (esto nos valdrá para predecir imágenes que no sean del dataset CIFAR10)

  gt = gt_str #(X)

  # Mostrando imagen e información
  label = "Predicción: {} - Confianza: {:.2f}% - Ground Truth: {}".format(label, proba[0][idx] * 100, gt)
  plt.imshow(output)
  plt.title(label)
  plt.show()





Mounted at /content/gdrive


In [ ]:

drive.mount('/content/gdrive')
imagetest ="/content/gdrive/My Drive/Curso_DL_CFP/Proyecto_Pokedex/charmander.jpg"

img_test = cv2.imread(imagetest, cv2.IMREAD_COLOR) # Leo imagen con OPENCV
img_test = cv2.cvtColor(img_test,cv2.COLOR_BGR2RGB) # Por defecto la carga en BGR, la convierto a RGB

# Muestro información de la imagen y hago la predicción sacando resultados
print(img_test.shape)
plt.imshow(img_test)
plt.title('my picture')
plt.show()

#escalo la imagen al mismo tamaño que las del modelo
img_test = cv2.resize(img_test, (32,32))


#if 'model' not in locals():
  drive.mount('/content/drive') #(X)
  model = model.load(modeloentrenado+'modelpokedex.h5')

# Predecimos la imagen pasando como parámetros la imagen, el modelo
predict_image(img_test, model, 'charmander') #(X)


IndentationError: ignored